In [15]:
# Parse.py
import numpy as np
from scipy.sparse import coo_array, lil_matrix
from scipy.sparse.linalg import splu
# ========================
# === Public functions ===
# ========================
def tpd_file2dict(fname):
    with open(fname, 'r') as f:
        s = f.read()
    d = _parse(s)
    # _checkparams(d)
    return d
# =====================================
# === Private functions and helpers ===
# =====================================
def _parse(s):
    snew = s.splitlines()
    snew = [line.split('#')[0] for line in snew] # Get rid of all comments
    snew = [line.replace('\t', '') for line in snew]
    snew = [line.replace(' ', '') for line in snew]
    snew = list(filter(len, snew))

    d = dict([line.split(':') for line in snew]) 
    return _parse_dict(d)
 

def _parse_dict(d):
       # Read/convert minimum required input and convert, else exit:
    d = d.copy()
    try:
        d['PROB_TYPE'] = d['PROB_TYPE'].lower()
        d['VOL_FRAC'] = float(d['VOL_FRAC'])
        d['FILT_RAD'] = float(d['FILT_RAD'])
        d['P_FAC'] = float(d['P_FAC'])
        d['NUM_ELEM_X'] = int(d['NUM_ELEM_X'])
        d['NUM_ELEM_Y'] = int(d['NUM_ELEM_Y'])
        d['NUM_ELEM_Z'] = int(d['NUM_ELEM_Z'])
        d['DOF_PN'] = int(d['DOF_PN'])
        d['ETA'] = str(d['ETA']).lower()

    except:
        raise ValueError('One or more parameters incorrectly specified.')

    # Check for number of iterations or change stop value:
    try:
        d['NUM_ITER'] = int(d['NUM_ITER'])
    except KeyError:
        try:
            d['CHG_STOP'] = float(d['CHG_STOP'])
        except KeyError:
            raise ValueError("Neither NUM_ITER nor CHG_STOP was declared")

    # Check for GSF penalty factor:
    try:
        d['Q_FAC'] = float(d['Q_FAC'])
    except KeyError:
        pass

    # Check for continuation parameters:
    try:
        d['P_MAX'] = float(d['P_MAX'])
        d['P_HOLD'] = int(d['P_HOLD'])
        d['P_INCR'] = float(d['P_INCR'])
        d['P_CON'] = float(d['P_CON'])
    except KeyError:
        pass

    try:
        d['Q_MAX'] = float(d['Q_MAX'])
        d['Q_HOLD'] = int(d['Q_HOLD'])
        d['Q_INCR'] = float(d['Q_INCR'])
        d['Q_CON'] = float(d['Q_CON'])
    except KeyError:
        pass

    # Check for active elements:
    try:
        d['ACTV_ELEM'] = _tpd2vec(d['ACTV_ELEM'], int) - 1
    except KeyError:
        d['ACTV_ELEM'] = _tpd2vec('', int)
    except AttributeError:
        pass

    # Check for passive elements:
    try:
        d['PASV_ELEM'] = _tpd2vec(d['PASV_ELEM'], int) - 1
    except KeyError:
        d['PASV_ELEM'] = _tpd2vec('', int)
    except AttributeError:
        pass

    # Check if diagonal quadratic approximation is required:
    try:
        d['APPROX'] = d['APPROX'].lower()
    except KeyError:
        pass

    # How to do the following compactly (perhaps loop through keys)? Check for
    # keys and create fixed DOF vector, loaded DOF vector and load values
    # vector.
    dofpn = d['DOF_PN']

    x = d.get('FXTR_NODE_X', '')
    y = d.get('FXTR_NODE_Y', '')
    z = d.get('FXTR_NODE_Z', '')
    d['FIX_DOF'] = _dofvec(x, y, z, dofpn)

    x = d.get('LOAD_NODE_X', '')
    y = d.get('LOAD_NODE_Y', '')
    z = d.get('LOAD_NODE_Z', '')
    d['LOAD_DOF'] = _dofvec(x, y, z, dofpn)

    x = d.get('LOAD_VALU_X', '')
    y = d.get('LOAD_VALU_Y', '')
    z = d.get('LOAD_VALU_Z', '')
    d['LOAD_VAL'] = _valvec(x, y, z)

    x = d.get('LOAD_NODE_X_OUT', '')
    y = d.get('LOAD_NODE_Y_OUT', '')
    z = d.get('LOAD_NODE_Z_OUT', '')
    d['LOAD_DOF_OUT'] = _dofvec(x, y, z, dofpn)

    x = d.get('LOAD_VALU_X_OUT', '')
    y = d.get('LOAD_VALU_Y_OUT', '')
    z = d.get('LOAD_VALU_Z_OUT', '')
    d['LOAD_VAL_OUT'] = _valvec(x, y, z)


    # The following entries are created and added to the dictionary,
    # they are not specified in the ToPy problem definition file:
    Ksize = d['DOF_PN'] * (d['NUM_ELEM_X'] + 1) * (d['NUM_ELEM_Y'] + 1) * \
    (d['NUM_ELEM_Z'] + 1) #  Memory allocation hint for PySparse
    d['K'] = lil_matrix((Ksize, Ksize)) #  Global stiffness matrix
    d['E2SDOFMAPI'] =  _e2sdofmapinit(d['NUM_ELEM_X'], d['NUM_ELEM_Y'], \
    d['DOF_PN']) #  Initial element to structure DOF mapping

    return d

def _tpd2vec(seq, dtype=float):
    """
    Convert a tpd file string to a vector, return a NumPy array.

    EXAMPLES:
        >>> _tpd2vec('1|13|4; 20; 25|28')
        array([  1.,   5.,   9.,  13.,  20.,  25.,  26.,  27.,  28.])
        >>> _tpd2vec('5.5; 1.2@3; 3|7|2')
        array([ 5.5,  1.2,  1.2,  1.2,  3. ,  5. ,  7. ])
        >>> _tpd2vec(' ')
        array([], dtype=float64)

    """
    finalvec = np.array([], dtype)
    for s in seq.split(';'):
        if s.count('|'):
            values = [dtype(v) for v in s.split('|')]
            values[1] += 1
            vec = np.arange(*values)
        elif s.count('@'):
            value, num = s.split('@')
            try:
                vec = np.ones(int(num)) * dtype(value)
            except ValueError:
                raise ValueError('%s is incorrectly specified' % seq)
        else:
            try:
                vec = [dtype(s)]
            except ValueError:
                vec = np.array([], dtype)
        finalvec = np.append(finalvec, vec)
    return finalvec

def _dofvec(x, y, z, dofpn):
    """
    DOF vector.

    """
    try:
        vec_x = _tpd2vec(x)
    except AttributeError:
        vec_x = np.array(x)

    try:
        vec_y = _tpd2vec(y)
    except AttributeError:
        vec_y = np.array(y)

    try:
        vec_z = _tpd2vec(z)
    except AttributeError:
        vec_z = np.array(z)

    dofx = (vec_x - 1) * dofpn
    dofy = (vec_y - 1) * dofpn + 1
    if dofpn == 2:
        dofz = []
    else:
        dofz = (vec_z - 1) * dofpn + 2
    return np.r_[dofx, dofy, dofz].astype(int)

def _valvec(x, y, z):
    """
    Values (e.g., of loads) vector.

    """
    try:
        vec_x = _tpd2vec(x)
    except AttributeError:
        vec_x = x

    try:
        vec_y = _tpd2vec(y)
    except AttributeError:
        vec_y = y

    if z:
        try:
            vec_z = _tpd2vec(z)
        except AttributeError:
            vec_z = z
    else:
        vec_z = []

    return np.r_[vec_x, vec_y, vec_z]

def _e2sdofmapinit(nelx, nely, dofpn):
    """
    Create the initial element to structure (e2s) DOF mapping (connectivity).
    Return a vector as a NumPy array.

    """
    if dofpn == 1:
        e2s = np.r_[1, (nely + 2), (nely + 1), 0]
        e2s = np.r_[e2s, (e2s + (nelx + 1) * (nely + 1))]
    elif dofpn == 2:
        b = np.arange(2 * (nely + 1), 2 * (nely + 1) + 2)
        a = b + 2
        e2s = np.r_[2, 3, a, b, 0, 1]
    elif dofpn == 3:
        d = np.arange(3)
        a = d + 3
        c = np.arange(3 * (nely + 1), 3 * (nely + 1) + 3)
        b = np.arange(3 * (nely + 2), 3 * (nely + 2) + 3)
        h = np.arange(3 * (nelx + 1) * (nely + 1), 3 * (nelx + 1) * (nely + 1) + 3)
        e = np.arange(3 * ((nelx+1) * (nely+1)+1), 3 * ((nelx+1) * (nely+1)+1) + 3)
        g = np.arange(3 * ((nelx + 1) * (nely + 1) + (nely + 1)),\
            3 * ((nelx + 1) * (nely + 1) + (nely + 1)) + 3)
        f = np.arange(3 * ((nelx + 1) * (nely + 1) + (nely + 2)),\
            3 * ((nelx + 1) * (nely + 1) + (nely + 2)) + 3)
        e2s = np.r_[a, b, c, d, e, f, g, h]
    return e2s




In [16]:
# Topology.py
# from parser import tpd_file2dict
MAX_ITERS = 250

SOLID, VOID = 1.000, 0.001 #  Upper and lower bound value for design variables
KDATUM = 0.1 #  Reference stiffness value of springs for mechanism synthesis

# Constants for exponential approximation:
A_LOW = -3 #  Lower restriction on 'a' for exponential approximation
A_UPP = -1e-5 #  Upper restriction on 'a' for exponential approximation

class Topology:
    def __init__(self, Es=1.0, vs=0.3) -> None:
        self.Es = Es
        self.vs = vs
        self.Gs = Es / (2*(1+vs))
    # ======================
    # === Public methods ===
    # ======================
    def load_tpd_file(self, fname):
        self.tpdfname = fname 
        self.topydict = tpd_file2dict(fname)

    def set_top_params(self):
        '''
        firstly, consider the general 3d case
        '''
        if not self.topydict:
            raise Exception('You must first load a TPD file!')
        self.probtype = self.topydict['PROB_TYPE'] #  Problem type
        self.probname = self.topydict.get('PROB_NAME', '') #  Problem name
        self.volfrac = self.topydict['VOL_FRAC'] #  Volume fraction
        self.filtrad = self.topydict['FILT_RAD'] #  Filter radius
        self.p = self.topydict['P_FAC'] #  'Standard' penalisation factor
        self.dofpn = self.topydict['DOF_PN'] #  DOF per node
        self.e2sdofmapi = self.topydict['E2SDOFMAPI'] #  Elem to structdof map
        self.nelx = self.topydict['NUM_ELEM_X'] #  Number of elements in X
        self.nely = self.topydict['NUM_ELEM_Y'] #  Number of elements in Y
        self.nelz = self.topydict['NUM_ELEM_Z'] #  Number of elements in Z
        self.fixdof = self.topydict['FIX_DOF'] #  Fixed dof vector
        self.loaddof = self.topydict['LOAD_DOF'] #  Loaded dof vector
        self.loadval = self.topydict['LOAD_VAL'] #  Loaded dof values
        # self.Ke = self.topydict['ELEM_K'] #  Element stiffness matrix
        self.K = self.topydict['K'] #  Global stiffness matrix

        # Check for either one of the following two, will take NUM_ITER if both
        # are specified.
        try:
            self.numiter = self.topydict['NUM_ITER'] #  Number of iterations
            
        except KeyError:
            self.chgstop = self.topydict['CHG_STOP'] #  Change stop criteria
            self.numiter = MAX_ITERS

        # All DOF vector and design variables arrays:
        if self.dofpn == 1:
            if self.nelz == 0: #  *had to this
                self.e2sdofmapi = self.e2sdofmapi[0:4]
                self.alldof = np.arange(self.dofpn * (self.nelx + 1) * \
                    (self.nely + 1))
                self.desvars = np.zeros((self.nely, self.nelx)) + self.volfrac
            else:
                self.alldof = np.arange(self.dofpn * (self.nelx + 1) * \
                    (self.nely + 1) * (self.nelz + 1))
                self.desvars = np.zeros((self.nelz, self.nely, self.nelx)) + \
                    self.volfrac
        elif self.dofpn == 2:
            self.alldof = np.arange(self.dofpn * (self.nelx + 1) * (self.nely + 1))
            self.desvars = np.zeros((self.nely, self.nelx)) + self.volfrac
        else:
            self.alldof = np.arange(self.dofpn * (self.nelx + 1) *\
                (self.nely + 1) * (self.nelz + 1))
            self.desvars = np.zeros((self.nelz, self.nely, self.nelx)) + \
                self.volfrac
        
        self.df = np.zeros_like(self.desvars) #  Derivatives of obj. func. (array)
        self.freedof = np.setdiff1d(self.alldof, self.fixdof) #  Free DOF vector
        self.r = np.zeros_like(self.alldof).astype(float) #  Load vector
        self.r[self.loaddof] = self.loadval #  Assign load values at loaded dof
        self.rfree = self.r[self.freedof] #  Modified load vector (free dof)
        self.d = np.zeros_like(self.r) #  Displacement vector
        self.dfree = np.zeros_like(self.rfree) #  Modified load vector (free dof)
        # Determine which rows and columns must be deleted from global K:
        self._rcfixed = np.where(np.in1d(self.alldof, self.fixdof), 0, 1)

    def fea(self):
        Kfree = self._updateK(self.K.copy())
        Kfree = Kfree.tocsc()
        lu = splu(Kfree)
        self.dfree = lu.solve(self.rfree)
        self.d[self.freedof] = self.dfree     

    def sens_analysis(self):
        self.objfval  = 0.0 #  Objective function value
        for elz in range(self.nelz):
            for elx in range(self.nelx):
                for ely in range(self.nely):
                    e2sdofmap = self.e2sdofmapi + self.dofpn *\
                                (ely + elx * (self.nely + 1) + elz *\
                                (self.nelx + 1) * (self.nely + 1))
                    if self.probtype == 'comp' or self.probtype == 'mech':
                        Ke = self._get_KE(elx,ely,elz)
                        DKe = self._get_KE_deriv(elx,ely,elz)
                        de = self.d[e2sdofmap]
                        
                        self.objfval += de @ Ke @ de
                        print(self.objfval)
                        self.df[elz, ely, elx] = -de @ DKe @ de


    def _updateK(self, K):
        """
        Update the global stiffness matrix by looking at each element's
        contribution i.t.o. design domain density and the penalisation factor.
        Return unconstrained stiffness matrix.

        """   
        for elz in range(self.nelz):
            for elx in range(self.nelx):
                for ely in range(self.nely):
                    e2sdofmap = self.e2sdofmapi + self.dofpn *\
                                (ely + elx * (self.nely + 1) + elz *\
                                (self.nelx + 1) * (self.nely + 1))
                    if self.probtype == 'comp' or self.probtype == 'mech':
                        updatedKe = self._get_KE(elx,ely,elz)
                        K[np.ix_(e2sdofmap, e2sdofmap)] = updatedKe.copy()




        # K.delete_rowcols(self._rcfixed) #  Del constrained rows and columns
        return K[np.ix_(self.freedof, self.freedof)]

    def _get_KE(self,elx,ely,elz):
        KE = np.zeros((24, 24))
        CE = self._get_CE(elx,ely,elz)
        GN_x = np.array([-1 / np.sqrt(3), 1 / np.sqrt(3)])
        GN_y = GN_x.copy()
        GN_z = GN_x.copy()
        GaussWeigh=[1, 1]
        dN = np.zeros((9, 24))
        for i in range(len(GN_x)):
            for j in range(len(GN_y)):
                for k in range(len(GN_z)):
                    x = GN_x[i]
                    y = GN_y[j]
                    z = GN_z[k]
                    dNx = 1/8*np.array([-(1-y)*(1-z),  (1-y)*(1-z),  (1+y)*(1-z), -(1+y)*(1-z), -(1-y)*(1+z),  (1-y)*(1+z),  (1+y)*(1+z), -(1+y)*(1+z)])
                    dNy = 1/8*np.array([-(1-x)*(1-z), -(1+x)*(1-z),  (1+x)*(1-z),  (1-x)*(1-z), -(1-x)*(1+z), -(1+x)*(1+z),  (1+x)*(1+z),  (1-x)*(1+z)])
                    dNz = 1/8*np.array([-(1-x)*(1-y), -(1+x)*(1-y), -(1+x)*(1+y), -(1-x)*(1+y),  (1-x)*(1-y),  (1+x)*(1-y),  (1+x)*(1+y),  (1-x)*(1+y)])
            
                    dN[0,0:24:3] = dNx
                    dN[1,0:24:3] = dNy
                    dN[2,0:24:3] = dNz

                    dN[3,1:24:3] = dNx
                    dN[4,1:24:3] = dNy
                    dN[5,1:24:3] = dNz

                    dN[6,2:24:3] = dNx
                    dN[7,2:24:3] = dNy
                    dN[8,2:24:3] = dNz
                    Be = dN
                    KE = KE + GaussWeigh[i] * GaussWeigh[j] * GaussWeigh[k] *(Be.T @ CE @ Be)
        print(KE)
        return KE
        
    def _get_CE(self,elx,ely,elz):
        p = self.desvars[elz, ely, elx]
        E,v,G = self._iso_moduli(p)
        C1111 = E * (1.0 - v) / (1.0 - v - 2*v**2)
        C1122 = (E * v) / (1.0 - v - 2*v**2)
        C1212 = G
        CE = [C1111,   0,     0,     0,   C1122,   0,     0,     0,   C1122, \
            0,   C1212,   0,   C1212,   0,     0,     0,     0,     0,  \
            0,     0,   C1212,   0,     0,     0,   C1212,   0,     0,  \
            0,   C1212,   0,   C1212,   0,     0,     0,     0,     0,  \
        C1122,   0,     0,     0,   C1111,   0,     0,     0,   C1122,\
            0,     0,     0,    0,     0,   C1212,   0,   C1212,   0,  \
            0,     0,   C1212,   0,     0,     0,   C1212,   0,     0,  \
            0,     0,     0 ,    0,     0,   C1212,   0,   C1212,   0,  \
        C1122,   0,     0,     0,   C1122,  0,     0,     0,   C1111]
        CE = np.array(CE).reshape((9,9))
        return CE

    def _iso_moduli(self, p):
        deriv = 0
        Es = self.Es
        vs = self.vs
        Gs = self.Gs

        E = Es * (( 2.05292e-01 - 3.30265e-02*vs) * (p**(1-deriv)) * (1+0*deriv) + 
	     ( 8.12145e-02 + 2.72431e-01*vs) * (p**(2-deriv)) * (1+1*deriv) +
	     ( 6.49737e-01 - 2.42374e-01*vs) * (p**(3-deriv)) * (1+2*deriv))

        v =( 2.47760e-01 + 1.69804e-02*vs) * (1-deriv) + \
	     (-1.59293e-01 + 7.38598e-01*vs) * (p**(1-deriv)) * (1+0*deriv) + \
	     (-1.86279e-01 - 4.83229e-01*vs) * (p**(2-deriv)) * (1+1*deriv) + \
	     ( 9.77457e-02 + 7.26595e-01*vs) * (p**(3-deriv)) * (1+2*deriv)

        G = Gs * (( 1.63200e-01 + 1.27910e-01*vs) * (p**(1-deriv)) * (1+0*deriv) + \
	     ( 6.00810e-03 + 4.13331e-01*vs) * (p**(2-deriv)) * (1+1*deriv) + \
	     ( 7.22847e-01 - 3.56032e-01*vs) * (p**(3-deriv)) * (1+2*deriv))
        
        return (E, v, G)

################# Deriv
    def _get_KE_deriv(self,elx,ely,elz):
        KE = np.zeros((24, 24))
        CE = self._get_CE_deriv(elx,ely,elz)
        GN_x = np.array([-1 / np.sqrt(3), 1 / np.sqrt(3)])
        GN_y = GN_x.copy()
        GN_z = GN_x.copy()
        GaussWeigh=[1, 1]
        dN = np.zeros((9, 24))
        for i in range(len(GN_x)):
            for j in range(len(GN_y)):
                for k in range(len(GN_z)):
                    x = GN_x[i]
                    y = GN_y[j]
                    z = GN_z[k]
                    dNx = 1/8*np.array([-(1-y)*(1-z),  (1-y)*(1-z),  (1+y)*(1-z), -(1+y)*(1-z), -(1-y)*(1+z),  (1-y)*(1+z),  (1+y)*(1+z), -(1+y)*(1+z)])
                    dNy = 1/8*np.array([-(1-x)*(1-z), -(1+x)*(1-z),  (1+x)*(1-z),  (1-x)*(1-z), -(1-x)*(1+z), -(1+x)*(1+z),  (1+x)*(1+z),  (1-x)*(1+z)])
                    dNz = 1/8*np.array([-(1-x)*(1-y), -(1+x)*(1-y), -(1+x)*(1+y), -(1-x)*(1+y),  (1-x)*(1-y),  (1+x)*(1-y),  (1+x)*(1+y),  (1-x)*(1+y)])
            
                    dN[0,0:24:3] = dNx
                    dN[1,0:24:3] = dNy
                    dN[2,0:24:3] = dNz

                    dN[3,1:24:3] = dNx
                    dN[4,1:24:3] = dNy
                    dN[5,1:24:3] = dNz

                    dN[6,2:24:3] = dNx
                    dN[5,2:24:3] = dNy
                    dN[8,2:24:3] = dNz
                    Be = dN
                    KE = KE + GaussWeigh[i] * GaussWeigh[j] * GaussWeigh[k] *(Be.T @ CE @ Be)
        return KE
        
    def _get_CE_deriv(self,elx,ely,elz):
        p = self.desvars[elz, ely, elx]
        E,v,G = self._iso_moduli(p)
        DE,Dv,DG = self._iso_moduli_deriv(p)
        C1111 = ((DE*(1-v)-E*Dv)*(1-v-2*v**2)-E*(1-v)*(-Dv-4*v*Dv)) / (1-v-2*v**2)**2
        C1122 = ((DE*v+E*Dv)*(1-v-2*v**2)-E*v*(-Dv-4*v*Dv)) / (1-v-2*v**2)**2
        C1212 = DG
        CE = [C1111,   0,     0,     0,   C1122,   0,     0,     0,   C1122, \
            0,   C1212,   0,   C1212,   0,     0,     0,     0,     0,  \
            0,     0,   C1212,   0,     0,     0,   C1212,   0,     0,  \
            0,   C1212,   0,   C1212,   0,     0,     0,     0,     0,  \
        C1122,   0,     0,     0,   C1111,   0,     0,     0,   C1122,\
            0,     0,     0,    0,     0,   C1212,   0,   C1212,   0,  \
            0,     0,   C1212,   0,     0,     0,   C1212,   0,     0,  \
            0,     0,     0 ,    0,     0,   C1212,   0,   C1212,   0,  \
        C1122,   0,     0,     0,   C1122,  0,     0,     0,   C1111]
        CE = np.array(CE).reshape((9,9))
        return CE

    def _iso_moduli_deriv(self, p):
        deriv = 1
        Es = self.Es
        vs = self.vs
        Gs = self.Gs

        E = Es * (( 2.05292e-01 - 3.30265e-02*vs) * (p**(1-deriv)) * (1+0*deriv) + 
	     ( 8.12145e-02 + 2.72431e-01*vs) * (p**(2-deriv)) * (1+1*deriv) +
	     ( 6.49737e-01 - 2.42374e-01*vs) * (p**(3-deriv)) * (1+2*deriv))

        v =( 2.47760e-01 + 1.69804e-02*vs) * (1-deriv) + \
	     (-1.59293e-01 + 7.38598e-01*vs) * (p**(1-deriv)) * (1+0*deriv) + \
	     (-1.86279e-01 - 4.83229e-01*vs) * (p**(2-deriv)) * (1+1*deriv) + \
	     ( 9.77457e-02 + 7.26595e-01*vs) * (p**(3-deriv)) * (1+2*deriv)

        G = Gs * (( 1.63200e-01 + 1.27910e-01*vs) * (p**(1-deriv)) * (1+0*deriv) + \
	     ( 6.00810e-03 + 4.13331e-01*vs) * (p**(2-deriv)) * (1+1*deriv) + \
	     ( 7.22847e-01 - 3.56032e-01*vs) * (p**(3-deriv)) * (1+2*deriv))
        
        return (E, v, G)
    
 




 


In [17]:
def optimise(topology):
# Optimising function:
    def _optimise(t):
        t.fea()
        t.sens_analysis()
        t.filter_sens_sigmund()
        t.update_desvars_oc()
    try:
        while topology.change > topology.chgstop:
            _optimise(topology)
    except AttributeError:
        for i in range(topology.numiter):
            _optimise(topology)
            

In [18]:
t = Topology()
t.load_tpd_file('mmb_beam_2d_reci.tpd')
t.set_top_params()
t.desvars
t.fea()
t.sens_analysis()


132.25082719873902
352.0696635240728
504.5798333498643
590.8740726734349
731.6064360784737
847.3560789508397
924.585043341935
1076.1028472426249
1205.5129011239592
1259.499492895485
1372.3555978511563
1460.1125600526718
1641.756164601714
1839.8752256249343
1878.6349418604436
2016.9242068266144
2212.6441223859783
2305.7063313078197
2439.90234768766
2590.6481381280073
2670.5568797009378
2742.2419819050606
2805.9599904393535
2823.916247254761


In [19]:
t.e2sdofmapi

array([ 3,  4,  5, 15, 16, 17, 12, 13, 14,  0,  1,  2, 63, 64, 65, 75, 76,
       77, 72, 73, 74, 60, 61, 62])

In [21]:
t.d


array([ 0.00000000e+00, -7.52308226e+00, -1.37180832e+01,  0.00000000e+00,
       -3.71185273e+00, -7.40858536e+00,  0.00000000e+00,  1.27841793e+01,
        1.66541217e+01,  0.00000000e+00,  2.74281893e+00, -1.42520503e+01,
        1.00356170e+01, -4.83412418e+00, -1.30286380e+01,  4.80833071e+00,
       -9.09907678e+00, -4.71608579e+00, -7.35545430e+00,  1.37736761e+01,
        2.35320772e+01, -1.94510372e+00,  1.72315825e-01, -7.82129979e+00,
       -2.51486430e+00, -1.53362052e+00,  1.04598626e+01,  2.84695757e+00,
       -7.20868883e+00,  1.94760101e+00, -1.00646462e+01, -5.62257439e+00,
        8.51788867e+00, -1.25803001e+01,  6.38056937e+00,  7.52850786e+00,
       -7.85745089e+00,  7.11848260e+00,  9.28053954e+00,  5.77587341e-01,
        6.99124126e+00, -2.77973231e+00,  1.06942559e+01, -8.84869022e+00,
       -2.82213011e+01, -6.59020294e-01,  5.80216558e-02,  2.08271462e+00,
        9.14443446e-01,  3.16954895e+00, -5.38403295e+00, -4.84520289e+00,
        7.68072243e+00, -